In [129]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [130]:
import os
import keras.backend as K

from elmo.lm_generator import LMDataGenerator, MTLLMDataGenerator
from elmo.model_girnet import ELMo

## Note
- fasttext: sent_vector: unsup: [0.5040783036202435, 0.44986425566245447]
- fasttext: sent_vector: sup: [0.5628058728541675, 0.5187983749157842]
- fasttext: lstm: unsup [0.6182707995419401, 0.5768815131210775] 
- fasttext: lstm: sup [0.6705, 0.6624]
- lstm 400,200: preplexity - 14 on cm_p_test_1000.txt
- lstm - mtl preplexity: 14.398013495954116, loss: 1.6848554463386536
- girnet: perplexity: 165.677334, losss

In [131]:
DATA_SET_DIR = '../twiter_scrapping/data/'

In [132]:
parameters = {
    'name': 'cm_girnet',
    'multi_processing': True,
    'n_threads': 10,
    'cuDNN': True if len(K.tensorflow_backend._get_available_gpus()) else False,
    'train_dataset': 'cm_p_train_small.txt',
    'valid_dataset': 'cm_p_test_1000.txt',
    'test_dataset': 'cm_test_main_1000.txt',
    'vocab': 'vocab',
    'vocab_size': 525133,
    'num_sampled': 8000,
    'charset_size': 262,
    'sentence_maxlen': 32,
    'token_maxlen': 50,
    'token_encoding': 'word',
    'epochs': 1,
    'patience': 2,
    'batch_size': 32,
    'clip_value': 1,
    'cell_clip': 5,
    'proj_clip': 5,
    'lr': 0.2,
    'shuffle': True,
    'n_lstm_layers': 1,
    'n_highway_layers': 1,
    'cnn_filters': [[1, 32],
                    [2, 32],
                    [3, 64],
                    [4, 128],
                    [5, 256],
                    [6, 512],
                    [7, 512]
                    ],
    'lstm_units_size': 400,
    'hidden_units_size': 200,
    'char_embedding_size': 16,
    'dropout_rate': 0.1,
    'word_dropout_rate': 0.05,
    'weight_tying': True,
    'unidirectional': True,
}



In [133]:
# train_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, 'en_p_train_small.txt'),os.path.join(DATA_SET_DIR, 'es_p_train_small.txt'),os.path.join(DATA_SET_DIR, 'cm_p_train_small.txt')],
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding']
# )
# val_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, 'en_p_test_1000.txt'),os.path.join(DATA_SET_DIR, 'es_p_test_1000.txt'),os.path.join(DATA_SET_DIR, 'cm_p_test_1000.txt')],
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding']
# )
test_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, parameters['test_dataset']),os.path.join(DATA_SET_DIR, parameters['test_dataset']),os.path.join(DATA_SET_DIR, parameters['test_dataset'])],
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding']
)

In [5]:
# train_en_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'en_p_train_small.txt'),
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding'])

# train_es_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'es_p_train_small.txt'),
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding'])

# train_cm_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'cm_p_train_small.txt'),
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding'])

test_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['test_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])
# val_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['valid_dataset']),
#                                 os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                 sentence_maxlen=parameters['sentence_maxlen'],
#                                 token_maxlen=parameters['token_maxlen'],
#                                 batch_size=parameters['batch_size'],
#                                 shuffle=parameters['shuffle'],
#                                 token_encoding=parameters['token_encoding'])

In [18]:
# Set-up Generators
train_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['train_dataset']),
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding'])

val_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['valid_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])

test_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['test_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])

In [134]:
# Compile ELMo
elmo_model = ELMo(parameters)
elmo_model.compile_elmo(print_summary=False)

In [7]:
# for tr in [train_en_generator, train_es_generator, train_cm_generator]:
#     elmo_model.train(train_data=tr, valid_data=val_generator)
#  5.5218
elmo_model.train(train_data=train_generator, valid_data=val_generator)

In [135]:
elmo_model.load_temp()

In [136]:
elmo_model.save(sampled_softmax=False, temp=False)

Language Model saved successfully


In [21]:
# cm_p_test_1000.txt
# vanilla :  46.75967671612614
# mtl: 39.42567465448776
# girnet: 165.677334
# cm_main_test_1000.txt
# vanilla :  73.61951568054178
# mtl: 58.723475872121696
# girnet: 132.512263
elmo_model.evaluate(test_generator)

In [ ]:
# Build ELMo meta-model to deploy for production and persist in disk
# elmo_model.wrap_multi_elmo_encoder(print_summary=True, save=True)

In [ ]:
# elmo_model.load()

In [ ]:
# Get ELMo embeddings to feed as inputs for downstream tasks
# elmo_embeddings = elmo_model.get_outputs(test_generator, output_type='word', state='mean')

# Generate 

In [10]:
import numpy as np
def sample_logits(preds, temperature=1.0):
    """

    Sample an index from a logit vector.

    :param preds:
    :param temperature:
    :return:
    """
    from scipy.misc import logsumexp
    preds = np.asarray(preds).astype('float64')

    if temperature == 0.0:
        return np.argmax(preds)

    preds = preds / temperature
    preds = preds - logsumexp(preds)

    choice = np.random.choice(len(preds), 1, p=np.exp(preds))

    return choice

In [23]:
batch = test_generator[0][0]

In [30]:
len(batch[2])

32

In [43]:
seed = np.array([2,3]).reshape(2,1,1)
elmo_model._model.predict([[[2,3]], seed, seed])

In [75]:
seed = [2,3]

In [76]:
temp_seed_forward = np.array(seed[:-1]+[0]).reshape(len(seed),1,1)

In [82]:
np.argmax(elmo_model._model.predict([np.array(seed).reshape(len(seed),1), temp_seed, temp_seed])[0])

6

In [ ]:
ls = seed.shape[0]

        # Due to the way Keras RNNs work, we feed the model a complete sequence each time. At first it's just the seed,
        # zero-padded to the right length. With each iteration we sample and set the next character.

        # tokens = np.concatenate([seed, np.zeros(size - ls)])
        tokens_all = []
        for i in range(out_num):
            np.array()
            tokens_all.append(np.concatenate([seed, np.zeros(size - ls)]))

        for i in range(ls, size):
            tokens_to_predict = []
            for j in range(out_num):
                tokens_to_predict.append(tokens_all[j][None, :])

            all_probs = self.model.predict(tokens_to_predict)

            # Extract the i-th probability vector and sample an index from it
            for j, probs in enumerate(all_probs):
                next_token = sample_logits(probs[0, i - 1, :], temperature=temperature)
                tokens_all[j][i] = next_token

                return [tokens.astype('int') for tokens in tokens_all]

In [108]:
vocab = test_generator.vocab

In [121]:
seed = [2, 6, vocab["what"], vocab["this"]]
size = 16
for i in range(len(seed), size):
    temp_seed_forward = np.array(seed[:-1]+[0]).reshape(len(seed),1,1)
    x_seed = np.array(seed).reshape(len(seed),1)
    pred = elmo_model._model.predict([x_seed, temp_seed_forward, temp_seed_forward])
    seed.append(np.argmax(pred[i-1]))

In [122]:
id2word = {}
for key in test_generator.vocab:
    id2word[test_generator.vocab[key]] = key

In [123]:
def decode(sents_index):
    ans = ""
    for index in sents_index:
        ans = ans +  id2word[index] + " "
    return ans

In [124]:
decode(seed)

'<bos> <user> what this is the sea <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> '

In [140]:
def generate(seed, size=32):
    for i in range(len(seed), size):
        temp_seed_forward = np.array(seed[:-1]+[0]).reshape(len(seed),1,1)
        x_seed = np.array(seed).reshape(len(seed),1)
        pred = elmo_model._model.predict([x_seed, temp_seed_forward, temp_seed_forward])
        seed.append(np.argmax(pred[i-1]))
    return seed

In [154]:
seed = [2,324]

In [155]:
        for i in range(len(seed), size):
            temp_seed_forward = np.array(seed[:-1] + [0]).reshape(len(seed), 1, 1)
            x_seed = np.array(seed).reshape(len(seed), 1)
            x = [x_seed, temp_seed_forward, x_seed, temp_seed_forward, x_seed, temp_seed_forward]
            pred = elmo_model._model.predict(x)
            seed.append(np.argmax(pred[2][i - 1]))

In [156]:
decode(seed)

'<bos> alguien <user> i am <user> i am <user> i am <user> i am <user> i '